In [1]:
import matplotlib.pyplot as plt
from sympy.physics.control.control_plots import matplotlib
%config InlineBackend.figure_format = 'retina'
%matplotlib notebook
import numpy as np
import nifty8 as ift
import pickle
from matplotlib.patches import Rectangle
import matplotlib.cm as cm
try:
    from gwpy.timeseries import TimeSeries
except:
    ! pip install -q "gwpy==3.0.9"
    ! pip install -q "matplotlib==3.7.3"
    ! pip install -q "astropy==6.1.4"
    from gwpy.timeseries import TimeSeries


def unpickle_me_this(filename: str, absolute_path=False):
    if absolute_path:
        file = open(filename, 'rb')
    else:
        file = open(filename, 'rb')
    data = pickle.load(file)
    file.close()
    return data

def pickle_me_this(filename: str, data_to_pickle: object):
    path = filename + ".pickle"
    file = open(path, 'wb')
    pickle.dump(data_to_pickle, file)
    file.close()


# -- Turn on interactive plotting
# plt.ion()

In [2]:
# -- Set a GPS time:
t0 = 1126259462.4    # -- GW150914
center = int(t0)  #-- Round GPS time to nearest second
#-- Choose detector as H1, L1, or V1
detector = 'H1'
strain = unpickle_me_this("GW150914_strain.pickle")
data = 1e19 * strain.value

In [3]:
import matplotlib.patches as patches

zero_time = 1126259446  # I got this zero time by looking at the caption of the figure produced by strain.plot().
time = np.array(strain.times) - zero_time  # in seconds
onset = t0 - zero_time


fig = plt.figure(figsize=(10,6))
plt.plot(time, data)
plt.title("'raw' strain of famous GW150914 event")
plt.text(onset+1, 7.5, "Hypothesized window of gravitational wave", color='g')
plt.ylim(-9, 9)
plt.vlines([onset - 0.3, onset + 0.3], -10, 10, color="green", lw=1)
plt.ylabel(r"Strain $[10^{-19}]$")
plt.xlabel("Time [s]", fontsize=12)
plt.show()

<IPython.core.display.Javascript object>

# Procedure

1. We take the big dataset over the 30 seconds, cut out the green window where the gravitational wave is hypothesized to be. This cuts the long data strip in small data strip 1 and 2.
2. We build windows in each smaller data strip of that overlap with each other 50% of the time to maximize the number of windows. The windows should enforce periodic boundary conditions onto the data for the fourier transform. We try to maximize the window maximal length to get a fine resolution in fourier space. At the same time, let's aim for at least 10 power spectra.
3. We fourier transform all windows, get the power spectra and build the average of all power spectra.

In [4]:
cond_1 = np.where(time < (onset-0.3))
cond_2 = np.where(time > (onset+0.3))
data_strip_1 = data[cond_1]
time_in_strip_1 = time[cond_1]
data_strip_2 = data[cond_2]
time_in_strip_2 = time[cond_2]

fig2 = plt.figure(figsize=(10,6))
plt.plot(time_in_strip_1, data_strip_1, label="'Data strip 1'")
plt.plot(time_in_strip_2, data_strip_2, label="'Data strip 2'")
plt.title("'raw' strain of famous GW150914 event")
plt.text(onset+1, 7.5, "Hypothesized window of gravitational wave", color='g')
plt.ylim(-9, 9)
plt.vlines([onset - 0.3, onset + 0.3], -10, 10, color="green", lw=3)
plt.ylabel(r"Strain $[10^{-19}]$")
leg = plt.legend(loc="upper left")
plt.xlabel("Time [s]", fontsize=12)
plt.show()

<IPython.core.display.Javascript object>

In [18]:
length_of_windows = 2

lf_edges_ds1 = np.arange(0, max(time_in_strip_1), length_of_windows/2)
r_edges_ds1 = lf_edges_ds1+length_of_windows

lf_edges_ds2 = np.arange(0, max(time_in_strip_2), length_of_windows/2)
r_edges_ds2 = lf_edges_ds2+length_of_windows

edges_ds1_too_far_right = np.where(r_edges_ds1 > onset)
r_edges_ds1 = np.delete(r_edges_ds1, edges_ds1_too_far_right)
lf_edges_ds1 = np.delete(lf_edges_ds1, edges_ds1_too_far_right)

edges_ds2_too_far_left = np.where(lf_edges_ds2 < onset + 0.3)

r_edges_ds2 = np.delete(r_edges_ds2, edges_ds2_too_far_left)
lf_edges_ds2 = np.delete(lf_edges_ds2, edges_ds2_too_far_left)

edges_ds2_too_far_right = np.where(r_edges_ds2 > max(time_in_strip_2))

r_edges_ds2 = np.delete(r_edges_ds2, edges_ds2_too_far_right)
lf_edges_ds2 = np.delete(lf_edges_ds2, edges_ds2_too_far_right)



In [25]:
fig3, ax = plt.subplots(figsize=(10,6))
plt.plot(time_in_strip_1, data_strip_1, label="'Data strip 1'", alpha=.1)
plt.plot(time_in_strip_2, data_strip_2, label="'Data strip 2'", alpha=.1)
plt.title("'raw' strain of famous GW150914 event")
plt.text(onset+1, 7.5, "Hypothesized window of gravitational wave", color='g')
plt.ylim(-9, 9)
plt.vlines([onset - 0.3, onset + 0.3], -10, 10, color="green", lw=3)
plt.ylabel(r"Strain $[10^{-19}]$")
leg2 = plt.legend(loc="upper left")


# Plot Rectangles to visualize
heights = [4, 4.5, 5] * len(lf_edges_ds1)
for i, (left, right) in enumerate(zip(lf_edges_ds1, r_edges_ds1)):
    width = right - left
    if i%2 != 0:
        c = (1,0,0)
        lw=1
        height = 4
    else:
        c= (0,0,1)
        lw=2
        height = 5
    rect = Rectangle((left, 0), width, heights[i], facecolor='none', linewidth=lw, edgecolor=c)
    ax.add_patch(rect)


# Plot Rectangles to visualize
for i, (left, right) in enumerate(zip(lf_edges_ds2, r_edges_ds2)):
    width = right - left
    if i%2 != 0:
        c = (1,0,0)
        lw=1
        height = 4
    else:
        c= (0,0,1)
        lw=2
        height = 5
    rect = Rectangle((left, 0), width, heights[i], facecolor='none', linewidth=lw, edgecolor=c)
    ax.add_patch(rect)

plt.text(-2,-4,f"Everything not in these windows, I will throw away.\n Number of windows: {len(lf_edges_ds1)+len(lf_edges_ds2)}", fontsize=12)
plt.xlabel("Time [s]", fontsize=12)
plt.show()


<IPython.core.display.Javascript object>

In [30]:
collection_of_small_datasets_strain = []
collection_of_small_datasets_times = []

for left_lim, right_lim in zip(lf_edges_ds1, r_edges_ds1):
    idcs = np.where((time > left_lim) & (time < right_lim))
    collection_of_small_datasets_strain.append(data[idcs])
    collection_of_small_datasets_times.append(time[idcs])

for left_lim, right_lim in zip(lf_edges_ds2, r_edges_ds2):
    idcs = np.where((time > left_lim) & (time < right_lim))
    collection_of_small_datasets_strain.append(data[idcs])
    collection_of_small_datasets_times.append(time[idcs])

Let's plot this so we are sure we can reconstruct the full set from the little window data

In [36]:

fig4 = plt.figure(figsize=(10,6))
for d, t in zip(collection_of_small_datasets_strain, collection_of_small_datasets_times):
    plt.plot(t, d)

plt.ylabel(r"Strain $[10^{-19}]$")
plt.xlabel("Time [s]", fontsize=12)
plt.show()

<IPython.core.display.Javascript object>

Now, we shall enforce artificial periodicity in each little data window..

In [38]:
from scipy.signal.windows import tukey

In [39]:
collection_of_small_datasets_strain_windowed = [el*tukey(M=len(el), alpha=0.1, sym=True) for el in collection_of_small_datasets_strain]

In [42]:
fig5 = plt.figure(figsize=(10,6))
plt.xlabel("Time [s]", fontsize=12)
plt.ylabel("Strain $[10^{-19}]$")

plt.plot(collection_of_small_datasets_times[0], collection_of_small_datasets_strain[0], label="Original data...")
plt.plot(collection_of_small_datasets_times[0], collection_of_small_datasets_strain_windowed[0], label="Tapered data...")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

Let's build the power spectrum.

In [43]:
def power_analyze_my_field(field_to_analyze, ht_operator):
    """
    For a stationary field, this function extracts k_lengths and the power spectrum.

    :param field_to_analyze:    A field defined over a regular grid space that contains the real space field values from which to extract the power spectrum.
    :param ht_operator:         The operator that transforms between the real space and harmonic space.
    :return:
    """
    ps = ift.power_analyze(ht_operator(field_to_analyze))
    return ps.domain[0].k_lengths, ps

n_dtps = len(collection_of_small_datasets_strain[0])
time_domain = ift.RGSpace((n_dtps, ), distances=length_of_windows/n_dtps)
time_domain_tuple = ift.DomainTuple.make(time_domain, )
HT = ift.HartleyOperator(time_domain_tuple)

window_fields = [ift.Field(time_domain_tuple, val=small_data) for small_data in collection_of_small_datasets_strain_windowed]


In [55]:
tupled_result = [power_analyze_my_field(el, HT) for el in window_fields]

In [60]:
k_lengths, power_spectrum_samples = zip(*tupled_result)
power_spectrum_domain = power_spectrum_samples[0].domain
k_length = k_lengths[0]
mean_power_spectrum_val = np.mean([pss.val for pss in power_spectrum_samples], axis=0)
power_spectrum = ift.Field(power_spectrum_domain, val=mean_power_spectrum_val)

In [65]:

fig6 = plt.figure(figsize=(10,6))

plt.plot(k_length, power_spectrum.val)
plt.loglog()
plt.vlines(10, 1e-11, 1, label="Something 'wrong' with the data happens left to this line. Let's ignore this for now", color="red")
plt.xlabel("Fourier mode $k$")
plt.ylabel("Power spectrum")
plt.title("Power spectrum from the data itself, assuming stationary noise outside of a known signal-containing window")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

Let's save the result

In [66]:
pickle_me_this("results_from_welch_averaging_data", [time_domain, k_length, power_spectrum])